Este script implementa un modelo de Ridge Regression con transformación polinomial de segundo grado aplicado a la predicción del valor máximo de Fire Radiative Power (FRP_max) en eventos de incendios forestales. A partir de esta predicción continua, se construye una clasificación operativa derivada que identifica eventos intensos mediante un umbral fijo de FRP ≥ 100 MW.

La incorporación de características polinomiales permite al modelo capturar relaciones no lineales suaves e interacciones simples entre variables climáticas, manteniendo al mismo tiempo la regularización propia de Ridge, lo que controla la complejidad y reduce el riesgo de sobreajuste. De esta forma, el modelo actúa como un baseline no lineal, situado entre la regresión lineal clásica y modelos no paramétricos más complejos.

Este enfoque permite evaluar de manera progresiva el impacto de la no linealidad en el desempeño predictivo y operativo del sistema, manteniendo una estructura metodológica clara y comparable con los modelos previamente analizados.

In [ ]:
# ============================================================
# MODELO 1B — RIDGE REGRESSION CON TRANSFORMACIÓN POLINOMIAL
# Clasificación derivada:
#   Evento intenso = 1 si FRP_max_predicho ≥ 100
# Dataset: Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv
# ============================================================
# Descripción general:
# Este modelo extiende Ridge Regression incorporando una transformación
# polinomial de grado 2 sobre las variables predictoras. El objetivo es
# capturar relaciones no lineales suaves e interacciones simples entre
# variables climáticas, manteniendo regularización explícita.
#
# El modelo sigue siendo una regresión + clasificación derivada y actúa
# como un baseline no lineal, intermedio entre Ridge lineal y Random Forest.
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge

from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    confusion_matrix,
    classification_report,
    precision_score,
    recall_score,
    f1_score
)

# -----------------------------
# 0) FUNCIÓN MATRIZ DE CONFUSIÓN (PALETA BLUES)
# -----------------------------
# Descripción:
# Función auxiliar para visualizar la matriz de confusión de la clasificación
# derivada. Se muestran explícitamente TN, FP, FN y TP, facilitando la lectura
# operativa del desempeño del modelo.

def plot_confusion_with_labels_blues(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()

    labels = np.array([
        [f"TN\n{tn}", f"FP\n{fp}"],
        [f"FN\n{fn}", f"TP\n{tp}"]
    ])

    fig, ax = plt.subplots(figsize=(6, 5))
    im = ax.imshow(cm, cmap="Blues")

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(["Pred 0", "Pred 1"])
    ax.set_yticklabels(["Real 0", "Real 1"])
    ax.set_xlabel("Predicción")
    ax.set_ylabel("Real")
    ax.set_title(title)

    thr = cm.max() / 2 if cm.max() > 0 else 0
    for i in range(2):
        for j in range(2):
            color = "white" if cm[i, j] > thr else "black"
            ax.text(j, i, labels[i, j], ha="center", va="center", color=color)

    plt.colorbar(im, ax=ax)
    plt.grid(False)
    plt.tight_layout()
    plt.show()


# -----------------------------
# 1) CARGA DEL DATASET
# -----------------------------
# Descripción:
# Se carga el dataset consolidado de eventos de incendio, que contiene
# información climática, FRP inicial, FRP máximo y la etiqueta real de riesgo.

DATA_PATH = "/content/sample_data/Dataset_Incendios_Eventos_TARGET_RIESGO_FRP.csv"
df = pd.read_csv(DATA_PATH)

print("✅ Dataset cargado")
print("Shape:", df.shape)
display(df.head())


# -----------------------------
# 2) CONFIGURACIÓN DE VARIABLES
# -----------------------------
# Descripción:
# Se definen explícitamente la variable objetivo de regresión, la etiqueta
# real de clasificación y el conjunto de variables predictoras utilizadas.

TARGET_REG = "frp_max"
TARGET_CLASS_REAL = "target_riesgo"
FRP_INICIAL_COL = "frp_inicial"

features = [
    "frp_inicial",
    "temperature_2m_mean", "relativehumidity_2m_mean",
    "windspeed_10m_mean", "precipitation_sum",
    "temperature_2m_mean_lag1", "temperature_2m_mean_lag2", "temperature_2m_mean_lag3",
    "relativehumidity_2m_mean_lag1", "relativehumidity_2m_mean_lag2", "relativehumidity_2m_mean_lag3",
    "windspeed_10m_mean_lag1", "windspeed_10m_mean_lag2", "windspeed_10m_mean_lag3",
    "precipitation_sum_lag1", "precipitation_sum_lag2", "precipitation_sum_lag3"
]

# Validaciones mínimas
for col in [TARGET_REG, TARGET_CLASS_REAL, FRP_INICIAL_COL]:
    if col not in df.columns:
        raise ValueError(f"❌ Falta columna requerida: {col}")

missing = [c for c in features if c not in df.columns]
if missing:
    raise ValueError(f"❌ Faltan columnas en features: {missing}")


# -----------------------------
# 3) ARMAR X e y (REGRESIÓN)
# -----------------------------
# Descripción:
# Se construyen las matrices de entrada (X) y salida (y) para el problema
# de regresión. Se eliminan filas inválidas para asegurar consistencia.

X = df[features].apply(pd.to_numeric, errors="coerce")
y_reg = pd.to_numeric(df[TARGET_REG], errors="coerce")

mask = y_reg.notna() & df[FRP_INICIAL_COL].notna()
X = X.loc[mask].reset_index(drop=True)
y_reg = y_reg.loc[mask].reset_index(drop=True)
y_class_real = df.loc[mask, TARGET_CLASS_REAL].astype(int).reset_index(drop=True)

print("\nDatos para modelar:")
print("X:", X.shape, "| y_reg:", y_reg.shape)


# -----------------------------
# 4) TRAIN / TEST SPLIT
# -----------------------------
# Descripción:
# Se divide el dataset en entrenamiento y prueba para evaluar el desempeño
# del modelo en datos no vistos.

X_train, X_test, y_train, y_test = train_test_split(
    X, y_reg,
    test_size=0.30,
    random_state=42
)


# -----------------------------
# 5) PIPELINE: RIDGE + POLINOMIAL + GRIDSEARCH
# -----------------------------
# Descripción:
# Se construye un pipeline que incluye:
# - Imputación de valores faltantes
# - Escalado de variables
# - Transformación polinomial de grado 2
# - Regresión Ridge
#
# Se utiliza GridSearchCV para seleccionar el nivel óptimo de regularización.

pipe_ridge_poly = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("ridge", Ridge(random_state=42))
])

param_grid = {
    "ridge__alpha": [1, 10, 100, 300, 1000]
}

gs = GridSearchCV(
    pipe_ridge_poly,
    param_grid,
    scoring="neg_mean_absolute_error",
    cv=5,
    n_jobs=-1,
    verbose=1
)

gs.fit(X_train, y_train)
model = gs.best_estimator_

print("\n✅ Ridge polinomial entrenado")
print("Mejor alpha:", gs.best_params_["ridge__alpha"])


# -----------------------------
# 6) MÉTRICAS DE REGRESIÓN
# -----------------------------
# Descripción:
# Se evalúa el desempeño del modelo en términos de regresión continua
# utilizando métricas estándar.

y_pred_test = model.predict(X_test)

print("\nMétricas regresión (test):")
print("MAE :", mean_absolute_error(y_test, y_pred_test))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_test)))
print("R²  :", r2_score(y_test, y_pred_test))


# -----------------------------
# 7) PREDICCIÓN FRP MAX PARA TODO EL DATASET
# -----------------------------
# Descripción:
# Se generan predicciones continuas para todos los eventos disponibles.
# Se aplica una restricción física para evitar valores negativos.

frp_max_pred = model.predict(X)
frp_max_pred = np.maximum(frp_max_pred, 0)


# -----------------------------
# 8) CLASIFICACIÓN DERIVADA POR FRP ≥ 100
# -----------------------------
# Descripción:
# Se transforma la predicción continua en una clasificación binaria
# basada en un umbral operativo fijo.
THR_FRP = 100.0
pred_intenso = (frp_max_pred >= THR_FRP).astype(int)


# -----------------------------
# 9) DATASET DE SALIDA
# -----------------------------
# Descripción:
# Se construye un dataset final que incorpora las predicciones del modelo
# y la clasificación derivada, manteniendo trazabilidad con los eventos.
df_out = df.loc[mask].copy().reset_index(drop=True)

df_out["frp_max_pred_ridge_poly"] = frp_max_pred
df_out["pred_intenso_frp100"] = pred_intenso
df_out["umbral_frp"] = THR_FRP

display(df_out[[
    "event_id_final",
    "frp_inicial",
    "frp_max",
    "frp_max_pred_ridge_poly",
    "pred_intenso_frp100",
    TARGET_CLASS_REAL
]].head())


# -----------------------------
# 10) MÉTRICAS DE CLASIFICACIÓN
# -----------------------------
# Descripción:
# Se evalúa el desempeño del modelo desde una perspectiva operativa,
# comparando las predicciones binarias con la etiqueta real de riesgo.

prec = precision_score(y_class_real, pred_intenso, zero_division=0)
rec  = recall_score(y_class_real, pred_intenso, zero_division=0)
f1   = f1_score(y_class_real, pred_intenso, zero_division=0)

print("\n================= MÉTRICAS CLASIFICACIÓN ================")
print(f"Umbral FRP_max_pred ≥ {THR_FRP}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1       : {f1:.3f}")

print("\nReporte de clasificación:")
print(classification_report(y_class_real, pred_intenso, digits=3, zero_division=0))


# -----------------------------
# 11) MATRIZ DE CONFUSIÓN
# -----------------------------
# Descripción:
# Visualización final de la matriz de confusión para analizar aciertos
# y errores del modelo Ridge polinomial.

plot_confusion_with_labels_blues(
    y_class_real,
    pred_intenso,
    title="Matriz de confusión — Ridge Polinomial (FRP_max_pred ≥ 100)"
)


# -----------------------------
# 12) GUARDAR DATASET (OPCIONAL)
# -----------------------------
# Descripción:
# Se exporta el dataset final con las predicciones del modelo para
# su uso posterior o comparación con otros enfoques.

df_out.to_csv(
    "Dataset_Ridge_Polinomial_FRP100.csv",
    index=False,
    sep=";",
    decimal="."
)


El modelo Ridge con transformación polinomial muestra una mejora respecto a la versión lineal, evidenciando que la incorporación de no linealidades suaves permite capturar parcialmente patrones adicionales presentes en los datos. Sin embargo, su desempeño sigue siendo limitado frente a enfoques no paramétricos como Random Forest, especialmente en la detección de eventos de incendio intensos (FRP ≥ 100).

Desde una perspectiva operativa, el modelo continúa presentando una baja sensibilidad (recall) para la clase de interés, lo que restringe su utilidad en escenarios de alerta temprana. Esto confirma que, aun con características polinomiales, las limitaciones estructurales de los modelos lineales regularizados persisten cuando se enfrentan fenómenos altamente no lineales, con colas pesadas y umbrales abruptos.

En consecuencia, el Ridge polinomial cumple adecuadamente su rol como modelo de referencia intermedio, validando que la simple incorporación de no linealidad no es suficiente para abordar plenamente la complejidad del problema. Este resultado refuerza la necesidad de utilizar modelos no lineales más flexibles, como Random Forest o redes neuronales, para aplicaciones orientadas a la detección de incendios severos.